In [129]:
env = DialogEnvironment()


# TODO
#actor = Actor(num_inputs, num_actions, args)
#critic = Critic(num_inputs, args)
#discrim = Discriminator(num_inputs + num_actions, args)
learning_rate = .001
l2_rate = .005
actor = Actor(hidden_size=2,num_layers=3)
critic = Critic(hidden_size=2,num_layers=3)
discrim = Discriminator(hidden_size=2,num_layers=3)

actor_optim = optim.Adam(actor.parameters(), lr=learning_rate)
critic_optim = optim.Adam(critic.parameters(), lr=learning_rate, 
                          weight_decay=l2_rate) 
discrim_optim = optim.Adam(discrim.parameters(), lr=learning_rate)

KeyboardInterrupt: 

Rough runthrough of GAIL for dialog generation, getting close...

In [130]:
from collections import deque  


In [133]:
#code for training 
import torch
import numpy as np


def subsample(data, target, n=15):
    return [x[::n] for x in data], [y[::n] for y in target]


def get_action(mu, std):
    action = torch.normal(mu, std)
    action = action.data.numpy()
    return action


def get_entropy(mu, std):
    dist = Normal(mu, std)
    entropy = dist.entropy().mean()
    return entropy

def log_prob_density(x, mu, std):
    log_prob_density = -(x - mu).pow(2) / (2 * std.pow(2)) \
                     - 0.5 * math.log(2 * math.pi)
    return log_prob_density.sum(1, keepdim=True)

def get_reward(discrim, state, action):
    """
    The reward function according to irl. It's log D(s,a). 
    
    Reward is higher the closer this is to 0, because the more similar it is to an expert action. :
    Is quite close to imitation learning, but hope here is that with such a large number of expert demonstrations and entropy bonuses etc. it learns more than direct imitation. 
    """
    state = torch.Tensor(state)
    try:
        action = torch.Tensor(action) # turn state into a tensor if not already
    except:
        pass
    state_action = torch.cat([state, action]) #TODO
    with torch.no_grad():
        return -math.log(discrim(state.resize(1,60,300),action.resize(1,60,300))[0].item())

def save_checkpoint(state, filename):
    return


def train_discrim(discrim, memory, discrim_optim, discrim_update_num, clip_param):
    """
    Training the discriminator. 

    Use binary cross entropy to classify whether or not a sequence was predicted by the expert (real data) or actor. 
    """
    memory = np.array(memory)  # s a r s' tuple
    states = torch.stack(list(memory[:, 0]))
    actions = torch.Tensor(np.array(actions)) #actions taken by actor/policy
    expert_actions = torch.stack(list(memory[:, 4])) #actions taken by the expert # TODO

    criterion = torch.nn.BCELoss() # classify

    for _ in range(discrim_update_num):

        learner = discrim(states, actions) #pass (s,a) through discriminator

        # TODO
       # demonstrations = torch.Tensor([states, expert_actions]) # pass (s,a) of expert through discriminator
        expert = discrim(states,expert_actions) #discrimator "guesses" whether or not these 
        # actions came from expert or learner
        discrim_loss = criterion(learner, torch.ones((states.shape[0], 1))) + \
                        criterion(expert, torch.zeros((states.shape[0], 1)))
                # discrim loss: predict agent is all wrong, get as close to 0, and predict expert is 1, getting as close to 1 as possible. 
        discrim_optim.zero_grad() # gan loss, it tries to always get it right. 
        discrim_loss.backward()
        discrim_optim.step()
            # take these steps, do it however many times specified. 

        expert_acc = ((discrim(states,expert_actions) < 0.5).float()).mean() #how often it realized the fake examples were fake
        learner_acc = ((discrim(states,actions) > 0.5).float()).mean() #how often if predicted expert correctly. 

    return expert_acc, learner_acc # accuracy, it's the same kind, but because imbalanced better to look at separately. 
 

In [131]:
def train_actor_critic(actor, critic, memory, actor_optim, critic_optim, actor_critic_update_num, batch_size, clip_param):
    """
    Take a PPO step or two to improve the actor critic model,  using GAE to estimate returns. 
    
    In our case each trajectory it most one step, so the value function will have to do. 
    
    
    """
    memory = np.array(memory) 
    # tuple of a regular old RL problem, but now reward is what the discriminator says. 
    states = np.vstack(memory[:, 0]) 
    actions = list(memory[:, 1]) 
    rewards = list(memory[:, 2])  #IRL Rewards
    masks = list(memory[:, 3]) 

    # compute value of what happened, see if what we can get us better. 
    old_values = critic(torch.Tensor(states))
    #GAE aka estimate of Value + actual return roughtly 
    returns, advants = get_gae(rewards, masks, old_values, gamma, lamda)
    
    # pass states through actor, get corresponding actions
    mu, std = actor(torch.Tensor(states))
    # new mus and stds? 
    old_policy = log_prob_density(torch.Tensor(actions), mu, std) # sum of log probability
    # of old actions

    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)

    for _ in range(actor_critic_update_num):
        np.random.shuffle(arr)

        for i in range(n // batch_size): 
            batch_index = arr[batch_size * i : batch_size * (i + 1)]
            #batch_index = torch.LongTensor(batch_index)
            
            inputs = torch.Tensor(states)[batch_index]
            actions_samples = torch.Tensor(actions)[batch_index]
            returns_samples = returns.unsqueeze(1)[batch_index]
            advants_samples = advants.unsqueeze(1)[batch_index]
            oldvalue_samples = old_values[batch_index].detach()
        
        
            values = critic(inputs) #
            clipped_values = oldvalue_samples + \
                             torch.clamp(values - oldvalue_samples,
                                         -clip_param, 
                                         clip_param)
            critic_loss1 = criterion(clipped_values, returns_samples)
            critic_loss2 = criterion(values, returns_samples)
            critic_loss = torch.max(critic_loss1, critic_loss2).mean()

            loss, ratio, entropy = surrogate_loss(actor, advants_samples, inputs,
                                         old_policy.detach(), actions_samples,
                                         batch_index)
            clipped_ratio = torch.clamp(ratio,
                                        1.0 - clip_param,
                                        1.0 + clip_param)
            clipped_loss = clipped_ratio * advants_samples
            actor_loss = -torch.min(loss, clipped_loss).mean()
            #print(actor_loss,critic_loss,entropy)
           # return actor_loss, critic_loss, entropy
            loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy #entropy bonus to promote exploration.

            actor_optim.zero_grad()
            loss.backward()
            actor_optim.step()

           # critic_optim.zero_grad()
           # loss.backward() 
            critic_optim.step()

def get_gae(rewards, masks, values, gamma, lamda):
    """
    How much better a particular action is in a particular state. 
    
    Uses reward of current action + value function of that state-action pair, discount factor gamma, and then lamda to compute. 
    """
    rewards = torch.Tensor(rewards)
    masks = torch.Tensor(masks)
    returns = torch.zeros_like(rewards)
    advants = torch.zeros_like(rewards)
    
    running_returns = 0
    previous_value = 0
    running_advants = 0

    for t in reversed(range(0, len(rewards))): #for LL, only ever one step :-)
        running_returns = rewards[t] + (gamma * running_returns * masks[t])
        returns[t] = running_returns

        running_delta = rewards[t] + (gamma * previous_value * masks[t]) - \
                                        values.data[t]
        previous_value = values.data[t]
        
        running_advants = running_delta + (gamma * lamda * \
                                            running_advants * masks[t])
        advants[t] = running_advants

    advants = (advants - advants.mean()) / advants.std()
    return returns, advants

def surrogate_loss(actor, advants, states, old_policy, actions, batch_index):
    """
    The loss for PPO. Re-run through network, recomput policy from states
    and see if this surrogate ratio is better. If it is, use as proximal policy update. It's very close to prior policy, but def better. 
    
    Not sure this actually works though. Should not the new mu and stds be used to draw,
    
        When do we use get_action? Only once in main, I think it should be for all? 
    """
    mu, std = actor(states)
    new_policy = log_prob_density(actions, mu, std)
    old_policy = old_policy[batch_index]

    ratio = torch.exp(new_policy - old_policy)
    surrogate_loss = ratio * advants
    entropy = get_entropy(mu, std)

    return surrogate_loss, ratio, entropy

In [132]:
memory = np.array(memory)  # s a r s' tuple
states = torch.stack(list(memory[:, 0]))
actions = torch.Tensor(np.array(actions)) #actions taken by actor/policy
expert_actions = torch.stack(list(memory[:, 4])) #actions taken by the expert # TODO

criterion = torch.nn.BCELoss() # classify

for _ in range(1):

    learner = discrim(states, actions) #pass (s,a) through discriminator

    # TODO
   # demonstrations = torch.Tensor([states, expert_actions]) # pass (s,a) of expert through discriminator
    expert = discrim(states,expert_actions) #discrimator "guesses" whether or not these 
    # actions came from expert or learner
    discrim_loss = criterion(learner, torch.ones((states.shape[0], 1))) + \
                    criterion(expert, torch.zeros((states.shape[0], 1)))
            # discrim loss: predict agent is all wrong, get as close to 0, and predict expert is 1, getting as close to 1 as possible. 
    discrim_optim.zero_grad() # gan loss, it tries to always get it right. 
    discrim_loss.backward()
    discrim_optim.step()


In [67]:
import sys
sys.path.append('../src')
from models import *
from dialog_environment import *
import torch.optim as optim


In [128]:
actor = Actor(hidden_size=2,num_layers=3)
critic = Critic(hidden_size=2,num_layers=3)
discrim = Discriminator(input_size = 300, hidden_size=1,device='cuda',num_layers=1)

Now the fake main...

In [81]:
episodes = 0
train_discrim_flag = True
total_sample_size = 10
max_iter_num = 1
for iter in range(max_iter_num):
    actor.eval(), critic.eval()
    memory = deque()

    steps = 0
    scores = []

    while steps < total_sample_size: 
        state, expert_action, raw_state, raw_expert_action = env.reset()
        state = state
        score = 0
            
        #print("breakpt")
       # break
        
        
        for _ in range(10000): 
            if render:
                print(raw_state, raw_expert_action)
            steps += 1

            #TODO
            mu, std = actor(state.resize(1,60,300))
            action = get_action(mu, std)[0]
            done= env.step(action)
            irl_reward = get_reward(discrim, state, action)
            if done:
                mask = 0
            else:
                mask = 1

            memory.append([state, action, irl_reward, mask,expert_action])

            sys.exit

            score += irl_reward

            if done:
                break

        episodes += 1
        scores.append(score)

    score_avg = np.mean(scores)
    print('{}:: {} episode score is {:.2f}'.format(iter, episodes, score_avg))

("There 's some more combustible fuel down in the storage lockers next to the lounge . I 'll go get it .  No , I do n't want us separated .  ",) You just sealed it off ; it ca n't get to that section . 
("Excellent ! <person> is my name . I 'm the most wanted man on <loc> <loc> <loc> . Except I 'm not on <loc> <loc> <loc> of course , more 's the pity .  ",) A common thief . 
("Hello ! Good morning .  They do n't see foreign women in this market . How much did you pay ?  ",) <quantity> <quantity> , <cardinal> , I suppose . Why ? 
("We been seein ' you all along the way . ",) Yeah . I been seein ' you , too . 
("True , but thanks to you , we know his weakness . We can keep him contained . It 's just a matter of time before we hunt him down . ",) Too bad you 're out of it . 
("Well , they do n't always leave with the ones they came in with .  ",) Big day <time> <time> . 
("No , <person> , I 'm swimming here . Then I 'm going home .  I guess what I 'm trying to say is -- I 'd like us to be

In [54]:
train_discrim(discrim, memory, discrim_optim)

TypeError: train_discrim() missing 4 required positional arguments: 'demonstrations', 'discrim_update_num', 'batch_size', and 'clip_param'

In [11]:
    actor.train(), critic.train(), discrim.train()
    if train_discrim_flag:
        expert_acc, learner_acc = train_discrim(discrim, memory, discrim_optim, demonstrations, args)
        print("Expert: %.2f%% | Learner: %.2f%%" % (expert_acc * 100, learner_acc * 100))
        if expert_acc > args.suspend_accu_exp and learner_acc > args.suspend_accu_gen:
            train_discrim_flag = False
    train_actor_critic(actor, critic, memory, actor_optim, critic_optim, args)

    if iter % 100:
        score_avg = int(score_avg)

        model_path = os.path.join(os.getcwd(),'save_model')
        if not os.path.isdir(model_path):
            os.makedirs(model_path)

        ckpt_path = os.path.join(model_path, 'ckpt_'+ str(score_avg)+'.pth.tar')

        save_checkpoint({
            'actor': actor.state_dict(),
            'critic': critic.state_dict(),
            'discrim': discrim.state_dict(),
            'z_filter_n':running_state.rs.n,
            'z_filter_m': running_state.rs.mean,
            'z_filter_s': running_state.rs.sum_square,
            'args': args,
            'score': score_avg
        }, filename=ckpt_path)

breakpt
0:: 0 episode score is nan


/scratch/nsk367/anaconda3/envs/irl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/nsk367/anaconda3/envs/irl/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'writer' is not defined

In [12]:
mu, std = actor(state.resize(1,60,300))

/scratch/nsk367/anaconda3/envs/irl/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [21]:
action = get_action(mu, std)[0]


In [23]:
action

array([[-0.01071095,  0.00190448, -0.0029541 , ...,  0.00138841,
        -0.01924055, -0.01198568],
       [-0.0139896 ,  0.00212001,  0.0004153 , ...,  0.00339333,
        -0.02670347, -0.02036267],
       [-0.01561475,  0.000129  , -0.00163433, ...,  0.00364156,
        -0.02750754, -0.0264746 ],
       ...,
       [-0.01550546, -0.00023765, -0.003465  , ...,  0.00744267,
        -0.02834896, -0.03748056],
       [-0.01525856, -0.00050071, -0.00495602, ...,  0.00845245,
        -0.02859554, -0.03642436],
       [-0.0150643 ,  0.00208199, -0.00483533, ...,  0.00793394,
        -0.02875101, -0.03694056]], dtype=float32)